In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
# import torchvision
# from torch.utils.tensorboard import SummaryWriter

In [2]:
ngpu = 1
# Decide si queremos correr en gpu o cpu
device = torch.device("cuda:0" if (torch.cuda.is_available() and ngpu > 0) else "cpu")
device

device(type='cuda', index=0)

In [3]:
ix_to_w = open('word_labels_freeling.txt', 'r', encoding='utf-8').read().splitlines()
ix_to_w[:6]

[',', 'de', 'que', 'el', 'la', 'a']

In [4]:
w_to_ix = {w: ix for ix, w in enumerate(ix_to_w)}
vocab_size = len(w_to_ix)
vocab_size

7612

In [5]:
class Model(nn.Module):
    def __init__(self, ngpu, D_in, D_emb, D_lstm, D_out):
        super(Model, self).__init__()
        self.ngpu = ngpu
        self.embedding = nn.Embedding(num_embeddings=D_in, embedding_dim=D_emb)#, padding_idx=0)
        self.lstm = nn.LSTM(input_size=D_emb, hidden_size=D_lstm) #, bias=True)#, batch_first=True)
        self.linear = nn.Linear(in_features=D_lstm, out_features=D_out) #, bias=True)

    def forward(self, sentence):
        T = len(sentence)

        embeddings = self.embedding(sentence).view(T, 1, -1)

        lstm_out, (ht, ct) = self.lstm(embeddings)
        lstm_out = lstm_out.view(T, -1)

        preact_out = self.linear(lstm_out).view(T, -1)

        return F.log_softmax(preact_out, dim=1)

In [7]:
# Dimensión de entrada (one-hot), tamaño del vocabulario
D_in = vocab_size

# Dimensión de la capa de embedding
D_emb = 64 # 32

# Dimensión de la capa lstm
D_lstm = 32 # 16

# Dimensión de la capa de salida
D_out = D_in

# Modelo por cargar
model_state = 'modelsaves/model-freeling-emb_64-lstm_32-seed_72-epochs_550'

In [8]:
model = Model(ngpu, D_in, D_emb, D_lstm, D_out)
model.load_state_dict(torch.load(model_state))
model.to(device)
model.eval()

Model(
  (embedding): Embedding(7612, 64)
  (lstm): LSTM(64, 32)
  (linear): Linear(in_features=32, out_features=7612, bias=True)
)

In [9]:
BOS = '<BOS>'
EOS = '<EOS>'
UNK = '<UNK>'

ixBOS = w_to_ix[BOS]
ixEOS = w_to_ix[EOS]
ixUNK = w_to_ix[UNK]

In [10]:
test_sents = [
    'andrés_manuel_lópez_obrador',
    'lópez_obrador',
    'ricardo_anaya',
    'anaya',
    'josé_antonio_meade',
    'meade',
    'el candidato'
]

In [11]:
test_sents_ix = [
    [ixBOS]+[w_to_ix[w] for w in sent.split()] 
    for sent in test_sents
]
test_sents_ix

[[7609, 64],
 [7609, 42],
 [7609, 51],
 [7609, 165],
 [7609, 46],
 [7609, 63],
 [7609, 3, 25]]

In [12]:
def to_pytorch_tensor(list_of_lists):
    return [
        torch.LongTensor(l).to(device)
        for l in list_of_lists
    ]

In [13]:
X_test = to_pytorch_tensor(test_sents_ix)
X_test

[tensor([7609,   64], device='cuda:0'),
 tensor([7609,   42], device='cuda:0'),
 tensor([7609,   51], device='cuda:0'),
 tensor([7609,  165], device='cuda:0'),
 tensor([7609,   46], device='cuda:0'),
 tensor([7609,   63], device='cuda:0'),
 tensor([7609,    3,   25], device='cuda:0')]

In [14]:
def print_ix_sentence(sentence, end='\n'):
    print(' '.join(ix_to_w[ix] for ix in sentence.data), end=end)

In [15]:
def generar_sent(sentence, limit = 47, choose_max=True):
    if len(sentence) == 0:
        sentence = torch.LongTensor([ixBOS]).to(device)
    torchEOS = torch.LongTensor([ixEOS]).to(device)
    i = 0
    prediccion = sentence[-1:]
    while not torch.eq(prediccion, torchEOS):
        out = model(sentence)
        if choose_max:
            prediccion = torch.argmax(out[-1:], dim=1)
        else:
            prediccion = torch.multinomial(torch.exp(out[-1]), 1)
#         print_ix_sentence(prediccion, end=' ') # Imprimo el siguiente caracter
        sentence = torch.cat((sentence, prediccion), dim=0) # Genero siguiente cadena
        i+=1
        if i>limit:
            break
    return sentence

In [16]:
with torch.no_grad():
    for sentence in X_test[:10]:
        print('===================================================')
        print('- Probando enunciado:')
        print_ix_sentence(sentence)
        print()
        
        print('- Enunciado generado:')
        gen_sentence = generar_sent(sentence, limit=60, choose_max=False)
        print_ix_sentence(gen_sentence)
        print('\n')

#         out = model(sentence)
#         prediccion = torch.argmax(out[-1:], dim=1)
#         print('- Texto generado:')
#         print_ix_sentence(prediccion)

- Probando enunciado:
<BOS> andrés_manuel_lópez_obrador

- Enunciado generado:
<BOS> andrés_manuel_lópez_obrador comentó que se llamó a el productor , como es su momento fundamental . <EOS>


- Probando enunciado:
<BOS> lópez_obrador

- Enunciado generado:
<BOS> lópez_obrador dijo que ya no se informó menos gobernador más colaboradores que en 30 legales para nuestra ayotzinapa . <EOS>


- Probando enunciado:
<BOS> ricardo_anaya

- Enunciado generado:
<BOS> ricardo_anaya lázaro_cárdenas estuvo acompañado en candidata de campaña saludó el encuentro con <UNK> , alto vive una comunicación : damián_zepeda , en nada un grupo entre todos , se va a pagar votos . <EOS>


- Probando enunciado:
<BOS> anaya

- Enunciado generado:
<BOS> anaya no está muy duarte . <EOS>


- Probando enunciado:
<BOS> josé_antonio_meade

- Enunciado generado:
<BOS> josé_antonio_meade congregó a meter con <UNK> . <EOS>


- Probando enunciado:
<BOS> meade

- Enunciado generado:
<BOS> meade se comprometió a que me están 